In [6]:
import os
import cv2
import numpy as np
import json
from skimage import filters
from scipy import ndimage

# 입력 및 출력 디렉토리 설정
input_dir = "/data/ephemeral/home/MCG/data/train/DCM"
output_dir = "/data/ephemeral/home/MCG/preProcessing"
label_dir = "/data/ephemeral/home/MCG/data/train/outputs_json"  # 라벨이 있는 디렉토리 경로 설정
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 가장 큰 외곽선 추출 함수
def extract_largest_contour(result_mask, original_image):
    contours, _ = cv2.findContours(result_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        contour_mask = np.zeros_like(result_mask)
        cv2.drawContours(contour_mask, [largest_contour], -1, 255, thickness=cv2.FILLED)
        return cv2.bitwise_and(original_image, original_image, mask=contour_mask)
    return result_mask

# 각 이미지를 처리하여 CLAHE 후 가장 큰 컨투어 추출 결과 저장
def process_image(image_path, output_path, label_path):
    image = cv2.imread(image_path)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    _, binary = cv2.threshold(gray_image, 45, 255, cv2.THRESH_BINARY)
    kernel = np.ones((8, 8), np.uint8)
    binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
    
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    largest_contour = max(contours, key=cv2.contourArea)
    hull = cv2.convexHull(largest_contour)
    
    convex_hull_mask = np.zeros_like(gray_image)
    cv2.drawContours(convex_hull_mask, [hull], -1, 255, thickness=cv2.FILLED)
    image_with_hull = cv2.bitwise_and(image, image, mask=convex_hull_mask)
    
    clahe = cv2.createCLAHE(clipLimit=3.2, tileGridSize=(42, 42))
    enhanced_image = clahe.apply(cv2.cvtColor(image_with_hull, cv2.COLOR_BGR2GRAY))
    
    # 히스토그램 평탄화 적용
    equalized_image = cv2.equalizeHist(enhanced_image)
    
    # Multi-Otsu thresholding 적용
    thresholds = filters.threshold_multiotsu(equalized_image, classes=4)
    brightest_region_mask_clahe = (equalized_image > thresholds[-1]).astype(np.uint8)
    
    kernel_dilate = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (20, 20))
    kernel_erode = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (4, 4))
    closed_mask_clahe = cv2.morphologyEx(brightest_region_mask_clahe, cv2.MORPH_ERODE, kernel_erode)
    closed_mask_clahe = cv2.morphologyEx(closed_mask_clahe, cv2.MORPH_DILATE, kernel_dilate)
    closed_mask_clahe = cv2.morphologyEx(closed_mask_clahe, cv2.MORPH_ERODE, kernel_erode)
    
    contour_result_clahe = extract_largest_contour(closed_mask_clahe * 255, image)

    # 라벨을 그리는 부분
    if os.path.exists(label_path):
        with open(label_path, 'r') as f:
            label_data = json.load(f)
        
        # 예를 들어, 라벨 데이터에서 좌표 가져오기
        if 'annotations' in label_data:
            for annotation in label_data['annotations']:
                if 'points' in annotation:
                    points = np.array(annotation['points'], dtype=np.int32)
                    cv2.polylines(contour_result_clahe, [points], isClosed=True, color=(0, 255, 0), thickness=1)
    
    cv2.imwrite(output_path, contour_result_clahe)

# 각 폴더를 탐색하여 이미지 처리 및 저장
for folder_name in os.listdir(input_dir):
    folder_path = os.path.join(input_dir, folder_name)
    if os.path.isdir(folder_path):
        for image_name in os.listdir(folder_path):
            if image_name.endswith(".png"):
                image_path = os.path.join(folder_path, image_name)
                label_path = os.path.join(label_dir, folder_name, f"{os.path.splitext(image_name)[0]}.json")
                
                hand_side = "right" if "R" in image_name else "left" if "L" in image_name else "unknown"
                
                output_filename = f"{folder_name}_{hand_side}_{image_name}"
                output_path = os.path.join(output_dir, output_filename)
                
                try:
                    process_image(image_path, output_path, label_path)
                    print(f"Processed and saved: {output_path}")
                except Exception as e:
                    print(f"Error processing file: {image_path}")
                    print(str(e))


Processed and saved: /data/ephemeral/home/MCG/preProcessing/ID521_unknown_image1667177691187.png


Processed and saved: /data/ephemeral/home/MCG/preProcessing/ID521_unknown_image1667177664643.png
Processed and saved: /data/ephemeral/home/MCG/preProcessing/ID038_unknown_image1661318857820.png
Processed and saved: /data/ephemeral/home/MCG/preProcessing/ID038_unknown_image1661318884439.png
Processed and saved: /data/ephemeral/home/MCG/preProcessing/ID008_unknown_image1661145313061.png
Processed and saved: /data/ephemeral/home/MCG/preProcessing/ID008_unknown_image1661145286692.png
Processed and saved: /data/ephemeral/home/MCG/preProcessing/ID034_unknown_image1661317748865.png
Processed and saved: /data/ephemeral/home/MCG/preProcessing/ID034_unknown_image1661317775801.png
Processed and saved: /data/ephemeral/home/MCG/preProcessing/ID327_unknown_image1664846512328.png
Processed and saved: /data/ephemeral/home/MCG/preProcessing/ID327_unknown_image1664846497621.png
Processed and saved: /data/ephemeral/home/MCG/preProcessing/ID345_unknown_image1664932998571.png
Processed and saved: /data/eph

In [ ]:
from PIL import Image

# 이미지 경로
image_path = "/data/ephemeral/home/MCG/data/groupKFold_seed21/fold1/train/Image/image1661144206667.png"

# 저장 경로
original_save_path = "original_image_2048x2048.png"
resized_save_path = "resized_image_512x512.png"

# 이미지 로드
original_image = Image.open(image_path)

# 원본 이미지 저장
original_image.save(original_save_path)
print(f"Original image saved at {original_save_path}")

# 512x512로 리사이즈
resized_image = original_image.resize((512, 512))

# 리사이즈된 이미지 저장
resized_image.save(resized_save_path)
print(f"Resized image saved at {resized_save_path}")


Original image saved at original_image_2048x2048.png
Resized image saved at resized_image_512x512.png


: 